In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision.transforms as transforms

from torch.nn import Module
from torch.nn import Conv1d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [126]:
class CNN(Module):
    def __init__(self, numChannels=20):
        # call the parent constructor
        super(CNN, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv1d(in_channels=numChannels, out_channels=10,
            kernel_size=15, padding=7)
        self.relu1 = ReLU()
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv1d(in_channels=10, out_channels=1,
            kernel_size=15, padding=7)
        self.relu2 = ReLU()
        # initialize our softmax classifier
        self.logSoftmax = LogSoftmax(dim=1)

    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

In [127]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

In [128]:
input = np.random.uniform(low=0, high=1, size=(100, 100, 20))
output = np.random.uniform(low=0, high=1, size=(100, 100))

In [129]:
input[1].shape

(100, 20)

In [134]:
class SequenceDataset(Dataset):
    def __init__(self, input, output, transform):
        self.input = input
        self.output = output
        self.transform = transform

    def __len__(self):
        return self.input.shape[0]

    def __getitem__(self, index):
        x = input[index].T
        y = output[index]

        if self.transform:
            x = self.transform(x)[0]

        return x, y

In [135]:
dataset = SequenceDataset(input, output, transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [90, 10])

train_loader = DataLoader(dataset=train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().double() # an error is thrown when .double() is not used.

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [136]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training
